# Named Entity Recognition

Задачи:

* Извлекать реплики с приветствием – где менеджер поздоровался. 
* Извлекать реплики, где менеджер представил себя. 
* Извлекать имя менеджера. 
* Извлекать название компании. 
* Извлекать реплики, где менеджер попрощался.
* Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


# Предобработка текста

In [1]:
import pandas as pd
import re
import nltk
import numpy as np
import pymorphy2

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

/Users/daivanov/opt/anaconda3/envs/ds_practicum_env/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
pip install pymorphy2
nltk.download('wordnet') 
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/daivanov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daivanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
data = pd.read_csv('test_data.csv')
data.head(5)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [4]:
# Напишем функцию позволяющею подготовить текст
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('russian'))
stopwords = stopwords.union(set(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', 'угу']))

def data_preparation(text):
    
    "используем регулярные выражения для очистки текста"
        
    text = re.sub(r'[^а-яА-Я]', ' ', text)
    text = text.lower()
    
    text = nltk.word_tokenize(text)
    text = [word for word in text if word not in stopwords]
    
    text_lemm = [lemmatizer.lemmatize(word) for word in text]
    
    clean_text = " ".join(text_lemm)
       
    return clean_text

data['clean_text'] = data['text'].apply(data_preparation)

# Извлекаем имя менеджера

In [6]:
data_manager = data.query('role == "manager"')
data_manager = data_manager.reset_index()

prob_thresh = 0.4
line_id = []
name = []

morph = pymorphy2.MorphAnalyzer()
for i in range(len(data_manager['clean_text'])):
    text = data_manager['clean_text'][i]
    
    for word in nltk.word_tokenize(text):
        for p in morph.parse(word):
            if 'Name' in p.tag and p.score >= prob_thresh:
                line_id.append(i)
                name.append(word)

In [7]:
data_manager = data_manager.drop('index', axis=1)

In [8]:
name = pd.Series(data=name, index=line_id, name="name")
name

1       ангелина
17      ангелина
46      ангелина
85       дмитрий
86        максим
87       дмитрий
87     анастасия
121    анастасия
178     вячеслав
181       максим
196      дмитрий
Name: name, dtype: object

In [9]:
df = data_manager.join(name, how='left')

In [10]:
df.loc[87]

,dlg_id,line_n,role,text,clean_text,name
87,3,4,manager,Да дмитрий вот мне моя коллега анастасия подск...,дмитрий коллега анастасия подсказала какие отк...,дмитрий
87,3,4,manager,Да дмитрий вот мне моя коллега анастасия подск...,дмитрий коллега анастасия подсказала какие отк...,анастасия


# Извлекаем реплики с приветствием – где менеджер поздоровался

In [11]:
greeting = ['здрав', 'добр', 'приве']
number_greeting = []   
for i in range(len(data_manager['text'])):
    for word in greeting:
        if word in data_manager['clean_text'][i] and data_manager['line_n'][i] < 3:
            number_greeting.append(i)
        else:
            pass

In [12]:
number_greeting

[0, 16, 45, 85, 86]

In [13]:
greeting_ser = pd.Series(data=['True','True', 'True', 'True', 'True'], index=number_greeting, name="greeting")
df = df.join(greeting_ser, how='left')

In [14]:
df.head(5)

,dlg_id,line_n,role,text,clean_text,name,greeting
0,0,1,manager,Алло здравствуйте,алло здравствуйте,NaN,True
1,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...,ангелина,NaN
2,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,возможно рассмотрите другие варианты видите хо...,NaN,NaN
3,0,8,manager,Угу а на что вы обращаете внимание при выборе,обращаете внимание выборе,NaN,NaN
4,0,11,manager,Что для вас приоритет,приоритет,NaN,NaN


# Извлекаeм реплики, где менеджер попрощался

In [15]:
bye_word = ['свидан', 'добр', 'скоро']
number_bye = []   
for i in range(len(data_manager['text'])):
    for word in bye_word:
        if word in data_manager['clean_text'][i] and data_manager['line_n'][i] > 3:
            number_bye.append(i)
        else:
            pass

In [16]:
bye_ser = pd.Series(data=['True','True', 'True', 'True', 'True'], index=number_bye, name="bye_word")
df = df.join(bye_ser, how='left')
df.tail(2)

,dlg_id,line_n,role,text,clean_text,name,greeting,bye_word
199,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,высылаю счет вами связи будут вопросы можете п...,NaN,NaN,NaN
200,5,142,manager,Ну до свидания хорошего вечера,свидания хорошего вечера,NaN,NaN,True


# Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [17]:
data_group = df.groupby('dlg_id')[['greeting', 'bye_word']].count()

In [18]:
data_group

,greeting,bye_word
dlg_id,,
0,1,1
1,1,1
2,1,0
3,2,1
4,0,1
5,0,1


In [19]:
for i in range(1, 6):
    if data_group['greeting'][i] != 0 and data_group['bye_word'][i] !=0:
        print(f'В диалоге {i} менеджер поздоровался и попрощался с клиентом')
    elif (data_group['greeting'][i] == 0 and data_group['bye_word'][i] !=0 or
    data_group['greeting'][i] != 0 and data_group['bye_word'][i] ==0):
        print(f'В диалоге {i} менеджер только поздаровался или порощался')
    else:
        print('Менеджер не поздаровался и не попращался')

В диалоге 1 менеджер поздоровался и попрощался с клиентом
В диалоге 2 менеджер только поздаровался или порощался
В диалоге 3 менеджер поздоровался и попрощался с клиентом
В диалоге 4 менеджер только поздаровался или порощался
В диалоге 5 менеджер только поздаровался или порощался


# Извлекать название компании

In [20]:
company = ['компани']
number_company = []
company_name=[]
for i in range(len(data_manager['text'])):
    for word in company:
        if word in data_manager['clean_text'][i]:
            number_company.append(i)
            
            t = data_manager['clean_text'][i]
            name = ' '.join(t.split(word)[1].split(' ')[1:3])# 3 words 
            company_name.append(name)
        else:
            pass


In [21]:
com_ser = pd.Series(data= company_name[0:4], index=number_company[0:4], name="company")
df = df.join(com_ser, how='left')
df.head(2)

,dlg_id,line_n,role,text,clean_text,name,greeting,bye_word,company
0,0,1,manager,Алло здравствуйте,алло здравствуйте,NaN,True,NaN,NaN
1,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...,ангелина,NaN,NaN,диджитал бизнес


# Извлекать реплики, где менеджер представил себя.

In [ ]:
Вероятнее всего менеджер представил себя в репликах, где есть 